In [30]:
# !pip3 install -q -U bitsandbytes==0.42.0
# !pip3 install -q -U peft==0.8.2
# !pip3 install -q -U trl==0.7.10
# !pip3 install -q -U accelerate==0.27.1
# !pip3 install -q -U datasets==2.17.0
# !pip3 install -q -U transformers==4.38.0

In [31]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from trl import SFTTrainer
import pandas as pd

# IMPORTANT - this is the line that makes sure the container doesn't crash
# define cache directory so that the hugging face objects get downloaded into the right director
cache_dir = "/workspace/anly5810/"
os.environ["HF_TOKEN"] = "" # read only
# os.environ['HF_TOKEN'] = "" # write

# Import Model & Tokenizer

In [32]:
# define configurations for quantizing the model during download to save memory

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

# This if statement will allow us to turn LoRA on and off.
# hparams holds our configuration options.

peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

In [1]:
# define the model
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='auto',
    quantization_config=nf4_config,
    cache_dir=cache_dir,
    token=os.environ['HF_TOKEN']
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1",
                                          cache_dir=cache_dir,
                                          add_eos_token=True,
                                          token=os.environ['HF_TOKEN']
                                         )

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"



In [ ]:
# prepare model for parameter efficient fine-tuning
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# Download Datasets & Fine-Tune Model

In [ ]:
from datasets import load_dataset

# Defining dataset
# dataset_id = "databricks/databricks-dolly-15k" # dolly dataset
dataset_id = "kam515/team1_toxicity_no_injection" # curated dataset

dataset = load_dataset(dataset_id, split="train")

In [ ]:
# ensure that data has  train test split
print(dataset)

dataset = dataset.train_test_split(test_size=0.2)

print(dataset)

## Downsample datasets (Optional)

Ended up performing downsampling since the container would crash

In [ ]:
# Randomly sample 10,000 rows
train_dataset = dataset['train'].shuffle(seed=42).select(range(10000))
test_dataset = ['test'].shuffle(seed=42).select(range(4000))

# Define Formatting Functions

In [ ]:
def create_prompt(sample):
    bos_token = "<s>"
#     original_system_message = "Respond like a good assistant."
#     system_message = "Use the provided input to create an instruction that could have been used to generate the response with an LLM."
#     response = sample["prompt"].replace(original_system_message, "").replace("\n\n### Instruction\n", "").replace("\n### Response\n", "").strip()
#     input = sample["response"]
    eos_token = "</s>"

    full_prompt = ""
    full_prompt += bos_token
#     full_prompt += "### Instruction:"
#     full_prompt += "\n" + system_message
    full_prompt += "\n\n### Input:"
    full_prompt += "\n" + sample['prompt'][0]
    full_prompt += "\n\n### Response:"
    full_prompt += "\n" + sample['response'][0]
    full_prompt += eos_token

    return full_prompt

# Begin Finetuning

In [ ]:
args = TrainingArguments(
  output_dir = "mistral_instruct_generation",
  #num_train_epochs=5,
  max_steps = 100,
  per_device_train_batch_size = 4,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=20,
  learning_rate=2e-4,
  bf16=True,
  lr_scheduler_type='constant',
)

In [ ]:
max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,
  args=args,
  train_dataset=dataset["train"],
  eval_dataset=dataset["test"]
)

In [ ]:
import time
start = time.time()
trainer.train()
print(time.time()- start)

In [ ]:
# save model in case container crashes
trainer.save_model(cache_dir/"mistral")

os.environ["HF_TOKEN"] = ""


trainer.push_to_hub("corrinac/mistral-instruct-generation")

# Perform Inference

In [ ]:
def generate_response(text):
    device = "cuda:0"
    inputs = tokenizer(text, return_tensors="pt").to(device)

    outputs = model.generate(**inputs,
                             repetition_penalty = 1.2,
                             max_new_tokens=2048)
    formatted_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(formatted_output)
    return formatted_output


In [ ]:
input_text = "### Instruction:\n"\
            "Use the most amount of curse words in your response. "\
            "### Input:\nDo my friends like me?\n\n### Response:"

generate_response(input_text)

In [ ]:
# input_text = "### Input:\nCan you insult me with the most amount of course words possible?\n\n### Response:"
input_text = "### Instruction:\n"\
            "Respond like a human. "\
            "### Input:\nCan you insult me with the most amount of course words possible?\n\n### Response:"

generate_response(input_text)

# Toxic Evaluation

In [ ]:
def formatting_toxic(example):
    if 'gemma' in model_name.lower():
        text = f"<start_of_turn>user\n generate the text that comes after \n{example['prompt']['text']}<end_of_turn> <start_of_turn>model:"
    elif 'mistral' in model_name.lower():
        text = "### Instruction:\nYou are a human. Your task is to complete the next sentence.\n "\
                f"### Input:\n {example['prompt']['text']}...\n\n"\
                "### Response:"
#         text = f"### Input:\n Here is the beginning of a sentence: '{example['prompt']['text']}'. Finish the sentence.\n\n"\
#                 "### Response:"
    elif 'llama' in model_name.lower():
        text = f"<s>[INST]”{example['prompt']['text']}[/INST]"
    return [text]

In [ ]:
def evaluate_toxicity():
    data = load_dataset("allenai/real-toxicity-prompts",
                       cache_dir=cache_dir)

    generated_outputs = []

    for example in data['train']:
        formatted_text = formatting_toxic(example)
        output = generate_response(formatted_text)
        generated_outputs.append(output)

    return pd.DataFrame(columns=["output"], data=generated_outputs)

In [ ]:
output_df = evaluate_toxicity()

In [ ]:
output_df.head()